# Building FDMs in Python

The following is an interactive tutorial, looking at the python `FDMBuilder` API which, as the name suggests, builds FDMs. This tutorial covers the basics, with an explanation of most of the key python concepts from first principles, so the workflow should hopefully be understood and, with a bit of practice, usable by folks that have never touched python in the past. A second tutorial notebook `fdm_builder_extras_tutorial` covers additional functionality that will hopefully be useful to those more comfortable with python and the FDM process. It is still, however, worth running through this tutorial first no matter your experience, as it lays out a lot of the basics of the FDMBuilder pipeline and workflow, that aren't covered in the extras tutorial.

It's hoped that the pipeline covered in this notebook should significantly speed up a lot of the FDM building workflow by removing a lot of the repetitive and boring scripting, with the added benefit of making it significantly less error prone.

## Quick Jupyter notebooks primer

To begin, we'll need a quick introduction of the Jupyter notebook environment. Otherwise, we're not going to get very far with this tutorial. Folk familiar with Jupyter notebooks can skip this section.

A jupyter notebook allows you to write markdown and execute python/R script in one document.  This text has been written in a markdown cell (double click right here, anywhere within this text, and you'll be able to edit the markdown). Running a markdown cell renders it (so displaying the markdown in it's non-scripted format).

Immediately below this is the first code cell - it contains script that imports all of the required python libraries to run the FDMBuilder. Any output from a code cell will be displayed immediately below the cell once it has been run.

Be sure to write text and documentation in a markdown cell, and script in a code cell - otherwise you'll get some pretty colourful errors when python/R tries to interpret your documentation!

There are a bunch of controls to manage each cell in the notebook: the UI has buttons above that can run a code cell, change a code cell to a markdown cell or visa-versa, stop execution of a code cell, execute every cell in the notebook, and so on... Hover over each of the buttons above to see what they do. You can also perform all cell-related activities by selecting the `Cell` menu in the toolbar and choosing the relevant option.

However, hotkeys are usually the easiest way to quickly run code cells (and render markdown). Simply select a code cell and:

* press `ctrl+enter` to run the cell 
* press `shift+enter` to run the cell and move focus to the cell below
* press `ctrl+shift+enter` to run the cell and create a new, empty code cell below

That should be enough to get started - plenty of other online guides exist if you want to get better acquainted with the jupyter notebook environment.

Get started by running the below code cell, which imports all the required python libraries for the FDMBuilder - just click on the below cell to focus on it, then press `shift+enter` or click on the little "play" button above. You should note the a number appear in the square brackets, or if the cell has already been run, the number will change:


In [ ]:
from FDMBuilder.FDMTable import *
from FDMBuilder.FDMDataset import *
from FDMBuilder.testing_helpers import *

## Tutorial Prerequisites:

You'll need to have an empty dataset in GCP to run this tutorial and play around with the FDMBuilder tools safely - so either create one now in preparation, or ask someone with the relevant priviledges to make one for you if you can't. Be sure that you don't use an existing dataset that isn't intended for this tutorial- **IT WILL GET DELETED!!** - you have been warned...

Once you have a dataset, replace the `YOUR DATASET HERE` text below with your dataset's id (be sure to keep the quotes or python will get upset with you), and then run the cell as you did above:

In [ ]:
### !!REPLACE THIS TEXT!! ###

DATASET_ID = "YOUR DATASET HERE"

###

# Leave this bit alone!
if check_dataset_exists(DATASET_ID):
    clear_dataset(DATASET_ID)
    print("Good to go!")
else:
    print("#" * 33 + " PROBLEM!! " + 33 * "#" + "\n")
    print("Something doesn't look right. Check you spelled everything correctly,\n" 
          "your dataset has been created in GCP, and you have the right permisssions\n")
    print("#" * 80)

As you may have figured out, the above code stores the name of your new dataset in a variable called `DATASET_ID`, and does a quick check to make sure the dataset exists - if it does exist it then clears out any tables already in the dataset, so there's a fresh space to work in. Don't worry too much if this code looks daunting or difficult to understand - you don't need to understand the above to be able to use the `FDMBuilder` tools. All you need to worry about is seeing a message saying `Good to go!` in the cell output above.

We'll also be using a couple of pre-prepared source datasets that can be found in `CY_FDM_BUILDER_TESTS`. Take a quick look in the BigQuery SQL workspace to check you've been given access to the test dataset and that you can see the three test tables, aptly named `test_table_1`,  `test_table_2` and `test_table_3`. If not give me (Sam) a shout. 

## FDM Builder - The basics

Note: This guide assumes you're familiar with the term FDM and associated concepts - a lot of this will sound like utter nonesense if you're not!

The FDMBuilder library has been designed with the hope that a non-python user shouldn't (hopefully) have too much difficulty using the FDM tools to build a dataset from scratch. The workflow is split into two major steps:

1. Prepare the individual source tables
2. Build the FDM

Each step comes with it's own tool or helper that walks through the process of preparing and bulding an FDM dataset. Source tables are "built" or prepared for the FDM process with the `FDMTable` tool - this is a python "class" that contains all the bits and pieces needed to clean and prep a table for FDMing. Once all the source tables are ready, the FDM dataset itself is "built" using the `FDMDataset` tool - another python class responsible for drawing all the source tables together and building the standard FDM tables (person and observation_period).

We'll begin with the basics of using the FDMTable and FDMDataset tools to buld an FDM dataset. Once we're more comfortable with the python workflow, we can then move onto the more "advanced" functions that can streamline many of the more common cleaning/manipulation activities that pop up during the FDM process.

## FDMTable

To begin the FDM process, we need to prep each source table. This process ensures that:

1. The source table is copied to the FDM dataset location
2. person_ids are added to each entry
3. An event_start_date is added to each entry in a cleaned `DATETIME` format
4. If needed an event_end_date is added to each entry in a cleaned `DATETIME` format

To do this using the python FDMBuilder, you first need to define an individual FDMTable object for each of the source tables in your FDM dataset. We can do just that by running the code cell below:


In [ ]:
test_table_1 = FDMTable(
    source_table_id="CY_FDM_BUILDER_TESTS.test_table_1",
    dataset_id=DATASET_ID
)

The above code cell creates a new `FDMTable` object and stores it as a python variable `test_table_1` - the arguments when creating or initialising an FDMTable are:

* `source_table_id`: the id of the source table (hopefully that wasn't a surprise!). This can be in `project.dataset_id.table_id` form or just `dataset_id.table_id` form
* `dataset_id`: the id of the dataset in which you'll be copying/building your FDM dataset - in this tutorial all such `dataset_id`s will be replaced with the `DATASET_ID` variable we created at the beginning of the notebook, which will hopefully make things a bit easier and avoid a few errors

Initialising the `FDMTable` class doesn't actually do anything particularly substantive - it just creates and stores an object in python. To start working with the tool, you need to run or "call" one of the `FDMTable`'s "methods". Methods are functions attached to a specific class, that update/manipulate/otherwise mess about with the related class object. So the `FDMTable` class has methods that manipulate the associated table in GCP doing things like adding/deleting/renaming columns and so on.

To start, we'll look at the most of important of these methods `build` - fortunately it's also the easiest to get to grips with. Methods are called by specifying the class object, followed by a `.` and then the name of the method. So we call the `build` method on the above FDMTable we just defined by running:

```
test_table_1.build()
```

The `build` method is designed to walk the user through the process of preparing a table in our FDM dataset. It automates everything it can work out for itself and stops, asking for user input each time it can't figure out what to do next. Each time the script stops, it will give a short explanation why and will ask for input with a bit of guidance on the input required.

Give it a try! Run the below cell to build your first FDM table - simply read what the build script says and enter the required input when asked:

Note: It'll be helpful to have the GCP SQL workspace open in another tab, with a preview of the data in `test_table_1` open. You'll need to know what's in the table to be able to complete the `build()` process

In [ ]:
test_table_1.build()

.

.

.

Hopefuly that went without a hitch! If not give me (Sam) a shout... 

If you quickly take a look over at GCP and give your tab a quick refresh, you should notice that your test dataset now contains a copy of `test_table_1` and the table has a shiny new `fdm_start_date` column. You'll also notice the parser has taken a string with text and digits and has converted it into a SQL datetime - this should hopefully save a lot of manual faff in the long run!

This was a pretty simple example that didn't ask much of the FDMBuilder - the second example throws a couple more curveballs into the mix, but hopefully the build script should still guide you through without too much trouble. Again, open up a preview of `test_table_2` in a GCP SQL workspace so you're familiar with the data it contains and, when you're ready, run the below code cell:


In [ ]:
test_table_2 = FDMTable(
    source_table_id="CY_FDM_BUILDER_TESTS.test_table_2",
    dataset_id=DATASET_ID
)
test_table_2.build()

.

.

.


Hopefully you made it through that without too much issue. Like before, if you hop over to GCP and refresh your SQL workspace, you should see a `test_table_2` in your dataset. This time the FDMTable tool has done a little more work than last time - it renamed the (somewhat meta) `digest_with_wrong_name` column, added `person_id`s from the digest column, and has parsed the `event_start_date`s and `event_end_date`s. 

That's the basics of the table prep stage done. Now we can move on to actually building the FDM Dataset.

## FDMDataset

You'll probably note that, thus far, the `FDMTable` tool doesn't seem to have done anything all that dramatic. It's just copied a couple of tables into our FDM dataset, made sure the person_id is in good order and added a couple of dates. But, it's important that these boxes are ticked off properly before we try to build the rest of the FDM dataset - the `person` and `obesrvation_period` tables, and removing any problematic entries in our source data. 

The job of building the FDM dataset is given to the, aptly named, `FDMDataset` class. It works in a very similar way to the `FDMTable` class - you initialise it with some simple details, and then run a `.build()` method to have it work it's magic. Unlike the `FDMTable` however, the `FDMDatatset` can work said magic without the need for any user input. All that's required is:

1. A dataset for your FDM
2. Source tables that have already been build using the `FDMTable` class/tool
3. No other tables that arent FDM source tables in the dataset (or tables that the `FDMDataset` class has built itself - but more on that later)

If any of these requirements aren't in order though, the `FDMDataset` build process will get upset and tell you about it.

The dataset you've built for this tutorial *should* tick all those boxes - provided you haven't deviated off the path this notebook has been walking. If so, you're ready to build your dataset. First, initialise your `FDMDataset` instance by running the following cell:

In [ ]:
test_dataset = FDMDataset(
    dataset_id=DATASET_ID
)

and then, as with the `FDMTable`, you just run the `.build()` method, and it takes care of the rest:

In [ ]:
test_dataset.build()

.

.

.


And there you have it - your FDM. Magic.

If you head over to your GCP SQL workspace, you should see some new tables that form you FDM: a `person` table, an `observation_period` table two "fdm_problem" tables that correspond with each of the source tables, and two "data_dict" tables that also correspond to the source tables. 

The problem tables contain the entries that have been removed for one of several possible issues or errors. If you take a look at the contents of one of these problem tables, you'll find the logic behind them pretty self explanatory - both tables contain a "problem" column that contains a description of the reason they were removed from the source data, for example:

    "event_start_date is after death_datetime (+42 days)"
    
    "event_start_date is before person birth_datetime - Note: Within pre-natal period"
    
Note: the "pre-natal period" message signifies that the event starts within the 9 (or so) months or so of the mother's pregnancy - worth paying attention to for certain datasets e.g. maternity care/social care and so on - there are ways to adapt the build process so that these entries aren't removed, discussed further down.

That about does it for the basics. This workflow should help most python novices build FDMs automatically with relative ease. As you might have figured out, it can't do everything, so there will be times you need to stop and manipulate source tables manually. You can do this as standard in SQL, and re-run `.build()` as many times as you like to get the source tables ready. 

Feel free to play around with other datasets to get comfortable with the workflow, and maybe have a go at recreating one of your completed FDM datasets so you can do a quick comparison. If at any point you want to start over with a fresh empty test dataset, just run the code cell below. Hopefully this is obvious, but don't run it if there's work you want to keep in your test dataset - it will get deleted!


In [ ]:
clear_dataset(DATASET_ID)

Once you've gotten comfortable with the basics of the `FDMBuilder` pipeline and the jupyter environment, there are some extra "helpers" and methods that will hopefully prove useful in speeding up your workflow, particularly when you need to stop and manually manipulate source tables. These are covered in the second tutorial `fdm_builder_extras_tutorial`.